In [1]:
eventsPath = os.environ["YAHOO_DATA"]
rawEventsRdd = sc.textFile(eventsPath + "events.txt")
rawEventsRdd.take(3)

[u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tair.com.smashatom.bingo\t1421009506\t47.237476\t-122.530884\tTacoma\t6\t12\tApp_Opened',
 u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tcom.android.vending\t1421029924\t47.237476\t-122.530891\tTacoma\t6\t18\tApp_Opened',
 u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tair.com.buffalo_studios.bingorush2\t1421015988\t47.237461\t-122.530899\tTacoma\t6\t14\tApp_Opened']

In [ ]:
rawAppListRdd = sc.textFile(eventsPath + "userapplist.txt")
rawAppListRdd.take(3)

In [ ]:
from collections import namedtuple

EventDataRow = namedtuple("EventDataRow", ["userId", "itemId", "ts", "latitude", "longitude", "city", "day_of_week", "time_of_day" , "event_type"])

def parseRawData(line):
    lineSplit = line.split("\t")
    return EventDataRow(userId=lineSplit[0],
                      itemId=lineSplit[1],
                      ts=int(lineSplit[2]),
                      latitude=float(lineSplit[3]),
                      longitude=float(lineSplit[4]),
                      city=lineSplit[5],
                      day_of_week=int(lineSplit[6]),
                      time_of_day=int(lineSplit[7]),
                      event_type=lineSplit[-1],
    )
    

eventsRdd = rawEventsRdd.map(parseRawData).cache()
eventsRdd.take(3)

In [ ]:
eventsRdd.filter(lambda x: x.city=="" ).take(10)

In [ ]:
userIdConversionDictionary = eventsRdd.map(lambda x: x.userId).distinct().zipWithIndex().collectAsMap()
userIdConversionDictionaryBroadcast = sc.broadcast(userIdConversionDictionary)
itemIdConversionDictionary = eventsRdd.map(lambda x: x.itemId).distinct().zipWithIndex().collectAsMap()
itemIdConversionDictionaryBroadcast = sc.broadcast(itemIdConversionDictionary)
cityConversionDictionary = eventsRdd.map(lambda x: x.city).distinct().zipWithIndex().collectAsMap()
cityConversionDictionaryBroadcast = sc.broadcast(cityConversionDictionary)

In [ ]:
eventsConvertedRdd = eventsRdd.map(lambda x: EventDataRow(
    userId=userIdConversionDictionaryBroadcast.value[x.userId],
    itemId=itemIdConversionDictionaryBroadcast.value[x.itemId],
    ts=x.ts,
    latitude=x.latitude,
    longitude=x.longitude,
    city=cityConversionDictionaryBroadcast.value[x.city],
    day_of_week=x.day_of_week,
    time_of_day=x.time_of_day,
    event_type=x.event_type
    ))

eventsConvertedRdd.take(3)

In [ ]:
eventsConvertedRdd.filter(lambda eventRaw: eventRaw.event_type=='App_Opened').map(lambda eventRaw: (
    eventRaw.userId,eventRaw.itemId,eventRaw.ts,eventRaw.city,eventRaw.day_of_week,eventRaw.time_of_day,
    eventRaw.latitude,eventRaw.longitude)
        ).saveAsTextFile(eventsPath + "events_parsed")

In [ ]:
import json
with open(eventsPath + 'userIdConversionDictionary.txt', 'w') as outfile:
    json.dump(userIdConversionDictionary, outfile)
with open(eventsPath + 'itemIdConversionDictionary.txt', 'w') as outfile:
    json.dump(itemIdConversionDictionary, outfile)
with open(eventsPath + 'cityConversionDictionary.txt', 'w') as outfile:
    json.dump(cityConversionDictionary, outfile)

In [ ]:
def parseUserAppList(line):
    lineSplit = line.split("\t")
    return userIdConversionDictionary[lineSplit[0]],[itemIdConversionDictionary[app[1:-1]] for app in lineSplit[1][1:-1].split(",")]

appListRdd = rawAppListRdd.map(parseUserAppList)
appListMap = appListRdd.collectAsMap()
with open(eventsPath + '/userAppMap.txt', 'w') as outfile:
    json.dump(appListMap, outfile)
appListRdd.take(3)

# Format data into context information

In [37]:
execfile("../script/utils.py")
eventRDD = loadDataset(eventsPath + "events_parsed_subset").groupBy(lambda x: x.userId).map(lambda (x,y): (x, sorted(list(y),key=lambda a: a.ts)))
#eventRDD = eventRDD.map(lambda x: (x[0], 
#                        map(lambda y : TrainRow(itemId=y.itemId, 
#                                               context = ContextRow(ts=y.ts,city=y.city,
#                                                                   lat=y.lat, lon=y.lon, moving = 1)), x[1])))

In [ ]:
def detectMovement(line):
    #location clustering
    listGroup = map(lambda x: list(x), line[1])
    workGroup = [x for x in listGroup if datetime.datetime.fromtimestamp(int(x[2])).hour >= 6 and  
            datetime.datetime.fromtimestamp(int(x[2])).hour <= 18]
    numNearLocation = []
    i = 0
    for x in workGroup:
        numNearLocation.append(0);
        for y in workGroup:
            if haversine(x[5], x[4], y[5], y[4]) < 0.1:
                numNearLocation[i] = numNearLocation[i] + 1
        i = i + 1
    if len(numNearLocation) > 0:
        index_work = numNearLocation.index(max(numNearLocation))
    else:
        index_work = -1
    
    homeGroup = [x for x in listGroup if datetime.datetime.fromtimestamp(int(x[2])).hour < 6 or
            datetime.datetime.fromtimestamp(int(x[2])).hour > 18]
    
    numNearLocation = []
    i = 0
    for x in homeGroup:
        numNearLocation.append(0);
        for y in homeGroup:
            if haversine(x[5], x[4], y[5], y[4]) < 0.1:
                numNearLocation[i] = numNearLocation[i] + 1
        i = i + 1
    if len(numNearLocation) > 0:
        index_home = numNearLocation.index(max(numNearLocation))
    else:
        index_home = -1

    if index_home != -1 and index_work != -1:
        listGroup = [(x[0],x[2],1) if haversine(x[5], x[4], workGroup[index_work][3], workGroup[index_work][2]) < 0.01
                 else( 
                    (x[0],x[2],2) if haversine(x[5], x[4], homeGroup[index_home][3], homeGroup[index_home][2]) < 0.01
                    else (x[0],x[2],3) 
                    )
                 for x in listGroup]
    else:
        listGroup = [(x[0],x[2],3)
                 for x in listGroup]
    
    
    listGroup = [(x[0],1) if datetime.datetime.fromtimestamp(int(x[1])).hour >= 6 and
                datetime.datetime.fromtimestamp(int(x[1])).hour <= 13
                    else(
                      (x[0],2) if datetime.datetime.fromtimestamp(int(x[1])).hour >= 13 and
                        datetime.datetime.fromtimestamp(int(x[1])).hour <= 18
                      else (x[0],3)
                    )
                for x in listGroup]
    #movement
    data = line[1]
    newData = [(data[0][1], data[0][2], data[0][3], data[0][4],data[0][5], 1, listGroup[0][1])]
    for i in xrange(1,len(data)):
        event = data[i]
        distance = haversine(event[5],event[4], data[i-1][5], data[i-1][4]) * 1000 #in meters
        time_difference = event.ts - newData[i-1][1] #in seconds
        moving = 1 #not available 
        if time_difference <= 300: #if 2 consecutive events are more than 300 seconds away, the movement is not available
            velocity =  distance/time_difference if time_difference > 0 else -1
            if velocity < 0:
                moving = 1; #not available
            elif velocity >= 0 and velocity <= 1:
                moving = 2  #standing still
            elif velocity <=2.4:
                moving = 3 #walking spead
            else:
                moving = 4 #faster
        newData.append((event[1],event[2],event[3],event[4],event[5], moving, listGroup[i][1]))
    #return (line[0], map(lambda el : TrainRow(el[0], ContextRow._make(el[1:])),newData))
    return (line[0], newData)
eventRDD_context = eventRDD.map(detectMovement)
eventRDD_context.take(1)
#train(itemId=60075, context=context(ts=1421371713, city=12940, lat=43.503536, lon=-88.558907, moving=1, location=3))

In [38]:
splitedData = splitRddV2(eventRDD_context,0.8)
splitedData.saveAsTextFile(eventsPath + "splitedData")


In [ ]:
train_rdd = sc.textFile(eventsPath + "train")
train_rdd = train_rdd.map(parseContextData)
train_rdd.take(1)

In [ ]:
test_rdd = sc.textFile(eventsPath + "test")
test_rdd = test_rdd.map(parseContextData)
test_rdd.take(1)

In [ ]:
splitedRD